# Personal project - "Applied data science capstone" course

## Introduction - Business problem

The objective of this project is to help citizens wanting to settle in a region knowing the infrastructures they will benefit from. By "infrastructure" we mean many different things such as historical site, restaurant, etc. 

For a given region (in this case, a French department - the example is given for the department "Nord"), and a given "infrastructure" (in this case, a "Historical site"), the notebook will provide the user with a map showing the cities for which there is at least one "infrastructure" located less than 1km from the city centre (geometrically speaking).

This kind of tool can be useful for several audiences :

- people looking for an accomodation in a region ;
- public decidors willing to check that all the region they are in charge of dispose from sufficient intrastructures ;
- *etc.*

## Data

We focus on France. Shapefiles are available at different disaggregation levels (region, departement, city, *etc.*) on [this website](https://gadm.org/download_country_v3.html).

As we are interested in finding cities where there are some given infrastructures, we download the most detailed shapefile, which is "gadm36_FRA_5_sf.rds". It gives for the 36000 french cities : the region, the departement, and the latitude and longitude of the corners of the cities. We preprocess this file to determine the "center" of each city. This preprocessing is done with R (code below). The prepared csv file is then imported to run the rest of the analysis (foursquare API).


*Code used to prepare the data with R*
```
# Import packages
library(data.table)

# Load data
sp_5 = readRDS("utils/gadm36_FRA_5_sf.rds")

# Loop over every line to find the "centre" of each city (mean of the coordinates)
for (i in 1:dim(sp_5)[1]){
  
  if (i %% 1000 == 0) print (i)
  sp_5$lng_centre[i] = mean(sp_5[i, "geometry"][[1]][[1]][[1]][, 1])
  sp_5$lat_centre[i] = mean(sp_5[i, "geometry"][[1]][[1]][[1]][, 2])
  
}
sp_5 = setDT(sp_5)

# Save transformed data
write.csv(sp_5, "sp_5_avec_centre.csv")
write.csv(sp_5[sp_5$NAME_2 == "Nord", !c("geometry")], "sp_5_avec_centre_nord.csv")
```


Note : we save also a subset of the data focusing on the "Nord" department in order to give a simple example in the notebook.

## Methodology

The methodology is very simple : this consists only in a descriptive analysis of a given region. So basically the main steps are :

- collecting georgraphical data (coordinates of the places of interest) ;
- requesting Foursquare API to find the relevant venues near our places of interest ;
- representing the results on a map.

## Code and results

In [117]:
# Importation of the libraries
import pandas as pd
import numpy as np
import folium 
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [114]:
# Importation of the shapefile data
data = pd.read_csv("sp_5_avec_centre_nord.csv", encoding = "latin1")
col = ['NAME_5', 'lng_centre', 'lat_centre']
data_short = data[col]

In [103]:
# Installation de la librairie Folium
!conda install -c conda-forge folium --yes

Solving environment: ...working... done

# All requested packages already installed.



We create a "base map" representing all the cities of the department : each one is represented by a blue circle. The Google API key is removed because the file is public. 

In [104]:
# We center the map on the centre of the department "Nord"
map_nord = folium.Map(location=[50.1434, 3.038688], zoom_start=8)
map_nord

# We make a loop on each point of the data to add the circle
for lat, lng, borough in zip(data['lat_centre'], data['lng_centre'], data['NAME_5']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nord)  

map_nord

Now we will use the Foursquare API to find the venues near our cities. Once again, the credentials are not shown here.

Then we will use the Foursquare api to find all the venues in a given area (with a radius of 1km). Then we will look into this list of venues to know if there is an historic site or not (it could easily be a restaurant or whatever other kind of venue). We create a dummy variable indicating the presence or not of an historic site.

In [109]:
# Creation of a blank variable
data_short.insert(loc = data_short.shape[1],
                  column = 'Site historique',
                  value = 0)

In [110]:
# We make a loop on every line of the geographical data (each city), find its coordinates in the data frame and launch the request of the Foursqaure API. Then we process the json file obtained to count if there is at least one historical site in the area.
for ligne in range(data.shape[0]):
    if (ligne%100 == 0) :
        print(ligne)
    # Coordonnées du centre de la ville
    neighborhood_latitude = data.loc[ligne, 'lat_centre']
    neighborhood_longitude = data.loc[ligne, 'lng_centre']

    # URL et requête
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id='+CLIENT_ID+'&client_secret='+CLIENT_SECRET+'&v='+VERSION+'&ll='+str(neighborhood_latitude)+','+str(neighborhood_longitude)+'&radius=1000&limit=100'
    results = requests.get(url).json()

    # Mise en forme des résultats de la requête et on regarde si on a un site historique dans la liste
    results = json_normalize(results['response']['groups'][0]['items'])

    types = []
    for indice in range(results.shape[0]):
        types.append(results.loc[indice, 'venue.categories'][0]['name'])

    if ('Historic Site' in types):
        data_short.loc[data['NAME_5'] == data.loc[ligne, 'NAME_5'], 'Site historique'] = 1



0


<ipython-input-110-046a80a948b6>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  results = json_normalize(results['response']['groups'][0]['items'])


100
200
300
400
500
600


In [116]:
# We count the number of cities with an historic site nearby
sum(data_short['Site historique'])

Finally we will represent on a map (with a red circle) the cities with a historic site.

In [112]:
# Map of the cities with an historic site
map_nord_site_histo = folium.Map(location=[50.1434, 3.038688], zoom_start=8)
map_nord_site_histo

# Same process than before to add the points on the map : loop on the lines of the filtered table
data_filtree = data_short[data_short['Site historique'] == 1]
for lat, lng, borough in zip(data_filtree['lat_centre'], data_filtree['lng_centre'], data_filtree['NAME_5']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.2,
        parse_html=False).add_to(map_nord_site_histo)  

map_nord_site_histo

This map represents the cities in the "Nord" department of France for which there is at least one "Historic Site" located less than 1km from the center of the city.

## Discussion - recommendations

One can note that the parameters can be easily changed : for example, the radius can be increased so that more cities will be considered as having a given venue in their neighborhood. One can also look for other kind of venues such as restaurants *etc.*. 

Given the map we obtain, we can see that there are not many cities with a historical site in this department : it could simplify an accomodation research for someone very much interested in cultural activities. Or it could help public decidors where to locate the next museum for example.

## Conclusion

This report presents a use-case example of the use of spatial data, combined with Foursquare data. Parameters can easily be tuned to adapt for the audience.